# Data Science Pandas - Joins, Time Series & Rolling Statistics

## Tasks Today:

1) <b>Pandas</b> <br>
 &nbsp;&nbsp;&nbsp;&nbsp; a) Joins <br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - merge() <br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - Breaking Down the Inner Join <br>
 &nbsp;&nbsp;&nbsp;&nbsp; b) Outer Joins <br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - Use Case Example <br>
 &nbsp;&nbsp;&nbsp;&nbsp; c) Rolling Statistics and Time Series w/Pandas <br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - read_csv() <br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - shift() <br>

In [12]:
import pandas as pd
import numpy as np

### Joins <br>
<p>Very often you will have to combine data from several different sources to obtain the actual dataset you need for your exploration or modeling. Pandas draws heavily on SQL in its API design for joins, and if you are familiar with SQL joins, then Pandas will come naturally. Imagine you have two tables, one with customer name and gender, and the other with their purchases:</p>

In [13]:
# Generate some fake data
city_data = {
    1:"Chicago",
    2:"Boston",
    3:"New York",
    4:"San Fransisco",
    5:"Washington D.C."
}

ages = np.random.randint(18,65,100)
city = [city_data[num] for num in np.random.randint(1,6,100)]
customer_id = np.arange(1000,1100)

<p>Let's assume we have a dataset containing the customer ID's cities, and ages for a set of customers. Suppose further, that we have a second table, containing some customer ID's, and a feedback survey rating from those customers. Suppose finally, that you as an analyst are asked the following question: What city tends to give the best feedback for our product? We'll need a join operation to get the answer.</p>

In [20]:
customer_demographics = pd.DataFrame.from_dict(
    {
        'ages': ages,
        'city': city,
        'customer_id': customer_id
    }
)
customer_demographics

,ages,city,customer_id
0,24,Chicago,1103
1,42,New York,1104
2,46,New York,1026
3,33,Chicago,1168
4,34,Washington D.C.,1057
5,19,Washington D.C.,1087
6,19,New York,1032
7,56,San Fransisco,1161
8,45,San Fransisco,1176
9,63,Boston,1184


In [21]:
# Generate some other fake data
customer_id = np.random.randint(1000,1200,100) # start, stop, how many
customer_rating = np.random.randint(1,11,100)

# Create survey dataframe from the customer data generated
surveys = pd.DataFrame.from_dict(
    {
        'customer_id':customer_id,
        'customer_rating': customer_rating
    
    }
)
surveys

,customer_id,customer_rating
0,1159,5
1,1136,3
2,1006,2
3,1044,10
4,1026,1
5,1153,8
6,1033,8
7,1045,1
8,1156,8
9,1091,9


##### merge()

<p>We can zip the two dataframes up with an inner join, using the Pandas API as follows:</p>

In [22]:
dem_ratings = customer_demographics.merge(surveys, on='customer_id',how='inner')
dem_ratings = dem_ratings.drop_duplicates(subset='customer_id',keep='first')
dem_ratings

,ages,city,customer_id,customer_rating
0,46,New York,1026,1
1,45,San Fransisco,1176,3
4,54,San Fransisco,1185,5
6,60,Chicago,1122,7
7,18,Washington D.C.,1189,6
8,59,Washington D.C.,1166,2
12,50,Washington D.C.,1009,9
13,62,New York,1128,2
15,49,Boston,1062,2
17,27,New York,1079,1


<p>Now it's a simple groupby operation to reveal the answer:</p>

In [27]:
dem_ratings.groupby('city',as_index=False).mean() #.sort_values(['customer_rating'], ascending=False)


,city,ages,customer_id,customer_rating
1,Chicago,58.666667,1115.000000,7.333333
4,Washington D.C.,48.285714,1101.857143,6.714286
3,San Fransisco,39.000000,1120.800000,5.800000
2,New York,43.750000,1092.125000,4.125000
0,Boston,35.750000,1086.750000,3.750000


##### Breaking Down the Inner Join <br>
<p>Let's take a closer look at the actual mechanics here however. There is some subtlety and nuance that is best understood by thinking of an inner join as two steps: a cartesian product, and then a filter. Observe the result of the following inner join:</p>

In [29]:
d = {
    'A':[1,2,3,4],
    'B':['a','b','c','d']
}

df1 = pd.DataFrame.from_dict(d)

df1

,A,B
0,1,a
1,2,b
2,3,c
3,4,d


In [31]:
d2 = {
    'A':[1,2,6,2],
    'C':['e','f','g','h']
}
df2 = pd.DataFrame.from_dict(d2)
df2

,A,C
0,1,e
1,2,f
2,6,g
3,2,h


In [32]:
df1.merge(df2, on='A', how='inner')

,A,B,C
0,1,a,e
1,2,b,f
2,2,b,h


<p>Somehow, we ended up with more rows in the result than either of the original tables! This is because we joined on a $\textit{non-unique index}$. The join starts by computing all the possible combinations of rows, and then filters them based on the condition. Because of the duplicates, multiple combinations of rows make it through!</p>

### Outer Joins <br>
<p>An $\textit{outer join}$ produces all of the rows from the left table, and joins whatever rows it can to the right table, filling anywhere it cant with NaN's or None values. Below is a brief example, and then we will discuss a possible use case for this type of join.</p>

In [37]:
df1 = pd.DataFrame.from_dict(
    {
        'A':[1,2,3],
        'B':['a','b','c']
    }
)
df2 = pd.DataFrame.from_dict(
    {
        'A':[1],
        'C':['d']
    }
)

In [38]:
df1.head()

,A,B
0,1,a
1,2,b
2,3,c


In [39]:
df2.head()

,A,C
0,1,d


In [40]:
df1.merge(df2, on = 'A', how = 'outer')

,A,B,C
0,1,a,d
1,2,b,NaN
2,3,c,NaN


##### Use Case Example

<p>As an example use case, let's imagine we have again the demographics and survey response tables from above:</p>

In [41]:
customer_demographics.head(5)

,ages,city,customer_id
0,24,Chicago,1103
1,42,New York,1104
2,46,New York,1026
3,33,Chicago,1168
4,34,Washington D.C.,1057


In [42]:
surveys.head(5)

,customer_id,customer_rating
0,1159,5
1,1136,3
2,1006,2
3,1044,10
4,1026,1


<p>Now, you are asked as an analyst to compare the demographics of survey responders to the demographics of all your customers combined.</p>

In [44]:
demo_surveys = customer_demographics.merge(surveys, on="customer_id",how="outer")
demo_surveys['customer_rating'] = demo_surveys['customer_rating'].fillna('N/A')
demo_surveys

,ages,city,customer_id,customer_rating
0,24.0,Chicago,1103,N/A
1,51.0,Washington D.C.,1103,N/A
2,42.0,New York,1104,N/A
3,46.0,New York,1026,1
4,33.0,Chicago,1168,N/A
5,18.0,Boston,1168,N/A
6,34.0,Washington D.C.,1057,N/A
7,23.0,San Fransisco,1057,N/A
8,19.0,Washington D.C.,1087,N/A
9,19.0,New York,1032,N/A


In [45]:
demo_surveys.groupby(['city','customer_rating']).size()

city             customer_rating
Boston           2.0                 2
                 3.0                 3
                 5.0                 2
                 8.0                 1
                 9.0                 1
                 N/A                19
Chicago          6.0                 1
                 7.0                 1
                 9.0                 1
                 N/A                 8
New York         1.0                 3
                 2.0                 1
                 3.0                 1
                 6.0                 1
                 7.0                 1
                 8.0                 2
                 9.0                 1
                 10.0                1
                 N/A                13
San Fransisco    1.0                 2
                 3.0                 2
                 5.0                 1
                 10.0                3
                 N/A                16
Washington D.C.  1.0           

In [46]:
# average age of people that answered the survey
demo_surveys[demo_surveys['customer_rating'] != 'N/A']['ages'].mean()

43.372093023255815

In [48]:
# find the average age of people that did not answer the survey
demo_surveys[demo_surveys['customer_rating'] == 'N/A']['ages'].mean()

40.985074626865675

### Rolling Statistics and Time Series with Pandas

##### read_csv()

In [49]:
weather = pd.read_csv('files/chiweather.csv',sep=';')

In [50]:
weather.head(10)

,Year,Month,Day,Hour,Minute,Temperature [2 m above gnd],Total Precipitation [sfc],Wind speed [10 m above gnd],Wind direction [10 m above gnd]
0,2017,11,25,0,0,51.85,0.0,17.31,327.13
1,2017,11,25,1,0,49.14,0.0,20.53,330.64
2,2017,11,25,2,0,46.42,0.0,21.41,327.80
3,2017,11,25,3,0,44.13,0.0,21.79,324.19
4,2017,11,25,4,0,42.22,0.0,21.03,321.91
5,2017,11,25,5,0,40.75,0.0,19.62,316.85
6,2017,11,25,6,0,40.71,0.0,19.17,318.31
7,2017,11,25,7,0,39.22,0.0,18.51,313.53
8,2017,11,25,8,0,39.49,0.0,18.73,310.16
9,2017,11,25,9,0,40.73,0.0,19.80,311.79


In [51]:
weather = weather.groupby(['Year','Month','Day']).mean().reset_index()

In [52]:
weather.head(50)

,Year,Month,Day,Hour,Minute,Temperature [2 m above gnd],Total Precipitation [sfc],Wind speed [10 m above gnd],Wind direction [10 m above gnd]
0,2017,11,25,11.5,0.0,41.208750,0.000000,17.939583,325.761667
1,2017,11,26,11.5,0.0,38.298333,0.000000,11.435417,258.533750
2,2017,11,27,11.5,0.0,45.611250,0.000000,11.082500,188.377083
3,2017,11,28,11.5,0.0,48.497500,0.000000,17.418750,244.673333
4,2017,11,29,11.5,0.0,40.122917,0.020833,13.107083,167.510000
5,2017,11,30,11.5,0.0,43.618750,0.033333,14.343333,276.060000
6,2017,12,1,11.5,0.0,41.860417,0.000000,6.845000,223.459583
7,2017,12,2,11.5,0.0,45.324167,0.000000,8.934167,218.252500


##### shift() <br>
<p>Shift index by desired number of periods with an optional time freq</p>

In [53]:
print(weather.keys())

Index(['Year', 'Month', 'Day', 'Hour', 'Minute',
       'Temperature  [2 m above gnd]', 'Total Precipitation  [sfc]',
       'Wind speed  [10 m above gnd]', 'Wind direction  [10 m above gnd]'],
      dtype='object')


In [57]:
weather['last_temp'] = weather['Temperature  [2 m above gnd]'].shift(1)
weather.head(5)

,Year,Month,Day,Hour,Minute,Temperature [2 m above gnd],Total Precipitation [sfc],Wind speed [10 m above gnd],Wind direction [10 m above gnd],last_temp
0,2017,11,25,11.5,0.0,41.208750,0.000000,17.939583,325.761667,NaN
1,2017,11,26,11.5,0.0,38.298333,0.000000,11.435417,258.533750,41.208750
2,2017,11,27,11.5,0.0,45.611250,0.000000,11.082500,188.377083,38.298333
3,2017,11,28,11.5,0.0,48.497500,0.000000,17.418750,244.673333,45.611250
4,2017,11,29,11.5,0.0,40.122917,0.020833,13.107083,167.510000,48.497500


In [61]:
# create a new column that takes the difference in temperature from the previous day as an absolute value
weather['dif_last_day'] = weather['last_temp'] - weather['Temperature  [2 m above gnd]'].shift(1)
weather.head(10)

,Year,Month,Day,Hour,Minute,Temperature [2 m above gnd],Total Precipitation [sfc],Wind speed [10 m above gnd],Wind direction [10 m above gnd],last_temp,dif_last_day
0,2017,11,25,11.5,0.0,41.208750,0.000000,17.939583,325.761667,NaN,NaN
1,2017,11,26,11.5,0.0,38.298333,0.000000,11.435417,258.533750,41.208750,0.0
2,2017,11,27,11.5,0.0,45.611250,0.000000,11.082500,188.377083,38.298333,0.0
3,2017,11,28,11.5,0.0,48.497500,0.000000,17.418750,244.673333,45.611250,0.0
4,2017,11,29,11.5,0.0,40.122917,0.020833,13.107083,167.510000,48.497500,0.0
5,2017,11,30,11.5,0.0,43.618750,0.033333,14.343333,276.060000,40.122917,0.0
6,2017,12,1,11.5,0.0,41.860417,0.000000,6.845000,223.459583,43.618750,0.0
7,2017,12,2,11.5,0.0,45.324167,0.000000,8.934167,218.252500,41.860417,0.0
